In [15]:
import gc
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
from pathlib import Path
from sklearn import svm
from sklearn.metrics import confusion_matrix, accuracy_score

In [16]:
def read_bl2(path):
    return np.core.records.fromfile(str(path),formats='i8,(2048)i2',names='index,wave')

In [17]:
def read_truth(path):
    return np.core.records.fromfile(str(path),formats='i8,i2,i2,(6)i2,i2,i2',names='index,label,numPro,naught,T0,max')

In [18]:
basepath = Path("/Users/elicarter/Local/FominResearch/RealData/Runs/pseudoData/sim50k_equalEvents_noWeirdTail_wfs.dat")
data = read_bl2(basepath)["wave"]
#CUT OFF AT 10K EVENTS
cutOff = 10000
trainingData = data[:cutOff] 
data = data[cutOff:]

In [19]:
truthTable = read_truth(Path("/Users/elicarter/Local/FominResearch/RealData/Runs/pseudoData/sim50k_equalEvents_noWeirdTail_proEventList.dat"))["label"]
trainingTruthTable = truthTable[:cutOff]
dataTruthTable = truthTable[cutOff:]

In [20]:
gc.collect()

120

# Relabel

Mark 1,2,3,8 as 0 and 0,4,5,6,7 as 1 to try and focus on finding them

In [21]:
for i in range(len(dataTruthTable)):
    if dataTruthTable[i] == 1 or dataTruthTable[i] == 2 or dataTruthTable[i] == 3 or dataTruthTable[i] == 8:
        dataTruthTable[i] = 0
    else:
        dataTruthTable[i] = 1
for i in range(len(trainingTruthTable)):
    if trainingTruthTable[i] == 1 or trainingTruthTable[i] == 2 or trainingTruthTable[i] == 3 or trainingTruthTable[i] == 8:
        trainingTruthTable[i] = 0
    else:
        trainingTruthTable[i] = 1

In [22]:
gc.collect()

40

# SVM

In [23]:
clf = svm.SVC()
clf.fit(trainingData, trainingTruthTable)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [26]:
filename = 'RareTrained_model.sav'
pickle.dump(clf, open(filename, 'wb'))
print(trainingData.shape)
print(data.shape)
print(trainingTruthTable.shape)

(10000, 2048)
(40000, 2048)
(10000,)


In [ ]:
labels = clf.predict(data)
print(accuracy_score(dataTruthTable, labels))

In [ ]:
df = pd.DataFrame(confusion_matrix(dataTruthTable, labels))

In [ ]:
df